In [1]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM, TimeDistributed
from keras.layers import Concatenate, Flatten
from keras.layers import GRU, Conv2D, MaxPooling2D
from keras.layers import Input, Reshape, Dot, Add
from keras.models import Model
from keras.optimizers import Adam
from keras.optimizers import RMSprop
# from keras.utils.vis_utils import plot_model
import keras
import keras.backend as K
from data_handle import *
from gensim_wrapper import *
from utils import *
import gensim
import random
import numpy as np
import tensorflow as tf
seed_val = 1000
random.seed(seed_val)
np.random.seed(seed_val)
tf.set_random_seed(seed_val)

c:\users\amany\appdata\local\conda\conda\envs\gputf3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
c:\users\amany\appdata\local\conda\conda\envs\gputf3\lib\site-packages\gensim\utils.py:1212: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
words = read_file(filename='data/news.txt')
unkown_word = "<unk>"
# words = [unkown_word] + words
xvocab, xword2int, xint2word = build_vocab(words)

words, word2freq = min_count_threshold(words)
vocab, word2int, int2word = build_vocab(words)
print(len(vocab))
# word2freq = get_frequency(words, word2int, int2word)
unigrams = [word2freq[int2word[i]] for i in range(len(word2int))]

char2int, int2char, char2tup, tup2char, n_consonant, n_vowel = build_charset()
ns_unigrams = ns_sample(word2freq, word2int, int2word, .75)
n_chars = 11 + 2 
n_features = len(char2int)
batch_size = 120
embed_size = 100
skip_window = 5

58383


In [3]:
def parseVec(file, delimiter):
    lines = open(file, encoding='utf8').readlines()
    vocab_size, embed_size = [int(s) for s in lines[0].split()]
    vocab_size  = min(len(word2int), vocab_size)
    embeddings = np.zeros((vocab_size + 1, embed_size), dtype=np.float64)
    for i in range(1, vocab_size):
        try:
            line = lines[i][:-1].split(delimiter)
            word = line[0]
            if 'unk' in word:
                continue
            if word in word2int:
                wordvec = np.array([np.float64(j) for j in line[1:] if j != ''])
                embeddings[word2int[word]] = wordvec
        except Exception as e:
            print(lines[i])
            print(e)
    return embeddings

In [4]:
del word2int['<unk>']

KeyError: '<unk>'

In [42]:
seq_emb = parseVec('results/w2v_high_2_5.txt', ' ')

In [43]:
emb = seq_emb
print(evaluate(word2int, normalize(emb), embed_size=emb.shape[1]))
print(evaluate(word2int, emb, embed_size=emb.shape[1]))

E:\Projects\amharic_word_embedding\data_handle.py:284: RuntimeWarning: invalid value encountered in true_divide
  return array / norms
c:\users\amany\appdata\local\conda\conda\envs\gputf3\lib\site-packages\gensim\matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int32 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


{'syntactic': 0.0, 'semantic': 0.0, 'total': 0.0, 'pick-one-out': 22.93577981651376}
{'syntactic': 0.0, 'semantic': 0.0, 'total': 0.0, 'pick-one-out': 21.10091743119266}


In [7]:
emb2D = np.zeros((seq_emb.shape[0], seq_emb.shape[1] * sem_emb.shape[1]))
for i in range(seq_emb.shape[0]):
    seq = seq_emb[i]
    sem = sem_emb[i]
    vec = np.outer(seq, sem).flatten()
    emb2D[i] = vec

In [39]:
lines = open('data/newan2.txt', encoding='utf-8').readlines()
emb = emb2D
correct = 0
for line in lines:
    if ':' in line:
        continue
    try:
        line = [x.strip() for x in line[:-1].split(' ')]
        vs = [emb[word2int[w]] for w in line]
        v = vs[1] - vs[0] + vs[2]
        sims = emb.dot(v)
        sim_with_word = [(word, sims[word2int[word]])
                             for word in word2int.keys()]
        sorted_sim = sorted(sim_with_word, key=lambda t: t[1], reverse=True)[:4]
#         print(type(sorted_sim[0][1]), sorted_sim[0][1])
        result = [pred[0] for pred in sorted_sim]
        if not np.isnan(sorted_sim[0][1]):
#             print(result, line[-1])
            if line[-1] in result:
#                 print(result, line[-1])
                correct += 1
    except:
        pass
print(correct * 100.0 / (len(lines) - 2))

0.9253903990746096


In [ ]:
# util = Utils(word2int, int2word, emb2D)
# lines = open('data/newan2.txt', encoding='utf-8').readlines()
# acc = np.array([0, 0])
# total = np.array([0, 0])
# ind = 0
# for line in lines[1:]:
#     if ':' in line[:-1]:
#         ind = 1
#         continue
#     ws = line[:-1].split()
#     result = util.solve(ws)
#     if result is None:
#         continue
#     if ws[-1] in result:
#         acc[ind] += 1
#     total[ind] += 1
# print(acc * 100 / total)

In [ ]:
# lines = open('data/analogies.txt', encoding='utf-8').readlines()
# e = normalize(cew)
# # e = cew
# for line in lines:
# #     ws = "ልጅቱ ልጁ ወይዘሮ አቶ".split(' ')
#     if ":" in line:
#         continue
#     try:
#         ws = line[:-1].split(' ')
#         v = normalize(vec(ws[1], e) - vec(ws[0], e) + vec(ws[2], e))
#         V = np.dot(e, v)
#         sort = list(sorted(enumerate(V[1:]), key=lambda x: x[1], reverse=True))
#         so = [int2word[index[0]] for index in sort[:4]]
#         if ws[-1] in so:
#             print(so, ws[-1])
# #         break
#     except:
#         pass

In [ ]:
# lines = open('data/analogies.txt', encoding='utf-8').readlines()
# words = []
# for line in lines:
#     ws = line[:-1].split(' ')
#     in_vocab = True
#     for w in ws:
#         if w not in word2int:
#             in_vocab = False
#         if in_vocab:
#             words.append(ws)
# ws = []
# for wl in words:
#     s = ' '.join(wl)
#     ws.append(s)
# ws = '\n'.join(ws)
# open('data/newan.txt', mode='w', encoding='utf-8').write(ws)

        
        

In [ ]:
# lines = open('data/analogies.txt', encoding='utf-8').readlines()
# words = []
# lns = {}
# for line in lines:
#     ws = line[:-1].split(' ')
#     in_vocab = True
#     for w in ws:
#         if w not in word2int:
#             in_vocab = False
#         if in_vocab:
#             ln = ' '.join(ws)
#             if ln not in lns:
#                 lns[ln] = len(lns)
#             ls = ' '.join(ws[2:4] + ws[0:2])
#             if ln not in lns:
#                 lns[ln] = len(lns)
# ws = '\n'.join(lns.keys())
# open('data/newan2.txt', mode='w', encoding='utf-8').write(ws)